# OCR del periódico *El Martillo* (Chiclayo, 1903–1919)

Este notebook procesa una página escaneada del periódico histórico peruano *El Martillo* empleando OCR con Google Gemini (API de Google AI Studio). El objetivo es extraer contenido estructurado, generar un dataset en CSV y visualizar la composición de la página.

## Flujo general
1. Carga de librerías
2. Configuración de API Key (Google AI Studio)
3. Descarga de imagen desde GitHub (sin subir archivos a Colab)
4. OCR con **Gemini 1.5 Flash**
5. Parseo del JSON
6. Exportación CSV
7. Visualización simple


In [ ]:
!pip install google-generativeai pandas pillow matplotlib requests

In [ ]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import google.generativeai as genai
import base64
import getpass
import requests
import io

## Configurar API Key (Google AI Studio)
La API Key nunca debe subirse a GitHub. Aquí se ingresa manualmente.

In [ ]:
api_key = getpass.getpass("Pega tu API Key de Google AI Studio: ")
genai.configure(api_key=api_key)

## Cargar la imagen directamente desde GitHub (RAW URL)
Así evitamos subir manualmente archivos a Colab.

In [ ]:
url = "https://raw.githubusercontent.com/ValLovaton/el-martillo-ocr-Pizarro_Sebastian_Lovaton_Valeria/main/data/el_martillo/page_01.png.jpg"

response = requests.get(url)
if response.status_code != 200:
    raise ValueError("No se pudo descargar la imagen desde GitHub")

img = Image.open(io.BytesIO(response.content))
img

## OCR con Google Gemini (modelo 1.5 Flash)
Solicitamos el contenido en formato JSON estructurado, siguiendo los requerimientos del curso.

In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash")

prompt = """
Eres un asistente especializado en OCR de periódicos históricos.
Extrae TODO el contenido legible de la página escaneada.

Devuelve únicamente una lista JSON, donde cada elemento tenga:
- date
- issue_number
- headline
- section
- type  (article / advertisement / other)
- text_excerpt

No incluyas texto fuera del JSON.
El JSON debe ser válido y parseable.
"""

image_bytes = response.content

response_gemini = model.generate_content([
    prompt,
    {"mime_type": "image/png", "data": image_bytes}
])

raw_output = response_gemini.text
raw_output

## Parsear JSON devuelto por Gemini
Incluye manejo de errores por si adiciona texto extra fuera del JSON.

In [ ]:
import re

try:
    structured = json.loads(raw_output)
except:
    matches = re.findall(r"\[.*\]", raw_output, re.DOTALL)
    if matches:
        structured = json.loads(matches[0])
    else:
        raise ValueError("Gemini no devolvió un JSON válido.")

df = pd.DataFrame(structured)
df

## Exportar dataset estructurado a CSV

In [ ]:
df.to_csv("page_01_structured.csv", index=False)
df.head()

## Visualización básica de los tipos de contenido

In [ ]:
df['type'].value_counts().plot(kind='bar', figsize=(6,4))
plt.title('Distribución de tipos de contenido en la página')
plt.xlabel('Tipo')
plt.ylabel('Cantidad')
plt.show()